In [2]:
import os

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import QuantileTransformer,  KBinsDiscretizer
from sklearn.model_selection import train_test_split
import warnings

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2021-09-18 23:18:13.375521: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
train.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1
3,3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1
4,4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1


In [4]:
train['nan_count'] = train.isna().sum(axis=1)
test['nan_count'] = test.isna().sum(axis=1)

features = [c for c in train.columns if c not in ('id', 'claim')]
pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median', missing_values=np.nan)),
    ('scaler', QuantileTransformer(n_quantiles=128, output_distribution='uniform')),
    ('bin', KBinsDiscretizer(n_bins=128, encode='ordinal', strategy='uniform'))
])
train[features] = pipe.fit_transform(train[features])
test[features] = pipe.transform(test[features])

xtrain = train[features]
ytrain = train['claim']
xtest = test[features]

#splliting the dataset into train and validation data
X_train, X_valid, y_train, y_valid = train_test_split(xtrain, ytrain, 
                                                    test_size=0.25,
                                                    random_state=0)

In [5]:
#centering the data
scaler = StandardScaler()
xtrain = scaler.fit_transform(X_train)
xvalid = scaler.transform(X_valid)
test = scaler.transform(xtest)

In [ ]:
input_shape = xtrain.shape[1:][0]

model = keras.Sequential([
    layers.Dense(input_shape, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.LayerNormalization(axis=-1),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.LayerNormalization(axis=-1),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')    
])

auc = keras.metrics.AUC(name='auc')
optimizer = keras.optimizers.Adam(lr = 1e-2, epsilon=1e-9, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics = [auc])
callback = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss', patience=5, restore_best_weights=True
            )
history = model.fit(x=np.float32(xtrain), 
          y=np.float32(y_train), 
          batch_size=1024, shuffle=True, 
          epochs=20,
          validation_data=(xvalid, y_valid),
          callbacks=[callback],
         )

2021-09-18 23:29:20.774830: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 341976964 exceeds 10% of free system memory.


Epoch 1/20
702/702 [==============================] - 5s 6ms/step - loss: 0.5654 - auc: 0.7602 - val_loss: 0.5154 - val_auc: 0.7998
Epoch 2/20
702/702 [==============================] - 4s 6ms/step - loss: 0.5154 - auc: 0.7979 - val_loss: 0.5147 - val_auc: 0.8015
Epoch 3/20
702/702 [==============================] - 3s 5ms/step - loss: 0.5144 - auc: 0.8000 - val_loss: 0.5139 - val_auc: 0.8030
Epoch 4/20
702/702 [==============================] - 4s 5ms/step - loss: 0.5138 - auc: 0.8006 - val_loss: 0.5138 - val_auc: 0.8039
Epoch 5/20
702/702 [==============================] - 4s 5ms/step - loss: 0.5125 - auc: 0.8024 - val_loss: 0.5138 - val_auc: 0.8045
Epoch 6/20
702/702 [==============================] - 4s 5ms/step - loss: 0.5121 - auc: 0.8033 - val_loss: 0.5140 - val_auc: 0.8047
Epoch 7/20
691/702 [============================>.] - ETA: 0s - loss: 0.5115 - auc: 0.8037

In [9]:
submission_sample = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')
submission_sample['claim'] = model.predict(np.float32(test))
submission_sample.to_csv('submission.csv', index=False)